# 4. Model Training and Comparison

In this notebook, we train multiple classification models to detect fraud and compare their performance using various metrics.

In [17]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from src.data.loading import load_fraud_data
from src.models.data_prep import prepare_model_data
from src.models.baseline import LogisticRegressionBaseline
from src.models.ensemble import RandomForestModel, XGBoostModel, LightGBMModel
from src.models.evaluation import calculate_metrics, stratified_kfold_cv, aggregate_cv_results, save_model
from src.models.comparison import compare_models, create_comparison_table, select_best_model
from src.visualization.model_viz import plot_confusion_matrix, plot_roc_curve, plot_precision_recall_curve, plot_feature_importance, plot_model_comparison

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## 1. Prepare Data

Loading the featured dataset and splitting into train/test sets with preprocessing.

In [18]:
# Load processed data (or re-load raw if pipeline is combined)
# Using raw load for demo purposes if processed isn't ready
df = pd.read_csv('../data/processed/fraud_featured.csv')

# Define columns
target = 'class'
drop_cols = ['user_id', 'signup_time', 'purchase_time', 'device_id', 'ip_address', 'country']

X_train, X_test, y_train, y_test = prepare_model_data(
    df,
    target,
    exclude_cols=drop_cols,
    test_size=0.2
)

2025-12-28 19:25:34 - fraud_detection - INFO - Starting complete data preparation pipeline...
2025-12-28 19:25:34 - fraud_detection - INFO - Separating features from target 'class'...
2025-12-28 19:25:34 - fraud_detection - INFO - Features: 21 columns
2025-12-28 19:25:34 - fraud_detection - INFO - Target: class
2025-12-28 19:25:34 - fraud_detection - INFO - Total samples: 129146
2025-12-28 19:25:34 - fraud_detection - INFO - Performing stratified train-test split (test_size=0.2)...
2025-12-28 19:25:34 - fraud_detection - INFO - Training set: 103316 samples
2025-12-28 19:25:34 - fraud_detection - INFO - Test set: 25830 samples
2025-12-28 19:25:34 - fraud_detection - INFO - Train class distribution:
class
0    0.90501
1    0.09499
Name: proportion, dtype: float64
2025-12-28 19:25:34 - fraud_detection - INFO - Test class distribution:
class
0    0.904994
1    0.095006
Name: proportion, dtype: float64
2025-12-28 19:25:34 - fraud_detection - INFO - Data preparation complete


## 2. Model Training & Evaluation

In [19]:
results = []

# --- 1. Baseline: Logistic Regression ---
print("Training Logistic Regression...")
lr = LogisticRegressionBaseline()
lr.train(X_train, y_train)
lr_metrics = lr.evaluate(X_test, y_test)
results.append(lr_metrics)

# --- 2. Random Forest ---
print("Training Random Forest...")
rf = RandomForestModel()
rf.train(X_train, y_train)
rf_metrics = rf.evaluate(X_test, y_test)
results.append(rf_metrics)

# --- 3. XGBoost ---
print("Training XGBoost...")
xgb = XGBoostModel()
xgb.train(X_train, y_train)
xgb_metrics = xgb.evaluate(X_test, y_test)
results.append(xgb_metrics)

# --- 4. LightGBM ---
print("Training LightGBM...")
lgbm = LightGBMModel()
lgbm.train(X_train, y_train)
lgbm_metrics = lgbm.evaluate(X_test, y_test)
results.append(lgbm_metrics)

Training Logistic Regression...
2025-12-28 19:25:35 - fraud_detection - INFO - Initialized Logistic Regression with params: {'max_iter': 1000, 'random_state': 42, 'class_weight': 'balanced'}
2025-12-28 19:25:35 - fraud_detection - INFO - Training Logistic Regression...


ValueError: could not convert string to float: 'Direct'

## 3. Comparison and Selection

In [ ]:
comparison_df = compare_models(results, sort_by='auc_pr')
comparison_table = create_comparison_table(results)
print(comparison_table)

best = select_best_model(results, primary_metric='auc_pr')
print(best['justification'])

In [ ]:
plot_model_comparison(comparison_df, metrics=['roc_auc', 'auc_pr', 'f1'])

## 4. Visualization for Best Model

Let's assume XGBoost or LightGBM performed best.

In [ ]:
y_pred = xgb.predict(X_test)
y_proba = xgb.predict_proba(X_test)

plot_confusion_matrix(y_test, y_pred)
plot_roc_curve(y_test, y_proba)
plot_precision_recall_curve(y_test, y_proba)
plot_feature_importance(xgb.model, X_train.columns)

## 5. Save the Best Model

In [ ]:
save_model(xgb.model, "best_fraud_model")
print("Best model saved to models/best_fraud_model.joblib")